#### In Machine Learning, Classification is used to identify the category new data belongs to basing on training data.
###### It is a supervised learning technique & labelled data is used
In this notebook, Support Vector Machine was used to build and train a model using human cell records with an aim of classifying cells to whether the samples are infectious or not.


In [ ]:
!pip install scikit-learn==0.23.1

In [1]:
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
%matplotlib inline 
import matplotlib.pyplot as plt

<h2>Load the Cancer data</h2>
The dataset consists of several hundred human cell sample records, each of which contains the values of a set of cell characteristics. The fields in each record are:

| Field name  | Description                 |
| ----------- | --------------------------- |
| ID          | Clump thickness             |
| Clump       | Clump thickness             |
| UnifSize    | Uniformity of cell size     |
| UnifShape   | Uniformity of cell shape    |
| MargAdh     | Marginal adhesion           |
| SingEpiSize | Single epithelial cell size |
| BareNuc     | Bare nuclei                 |
| BlandChrom  | Bland chromatin             |
| NormNucl    | Normal nucleoli             |
| Mit         | Mitoses                     |
| Class       | Benign or malignant         |


In [2]:
# To download the data
#Click here and press Shift+Enter
# !wget -O cell_samples.csv https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%203/data/cell_samples.csv

## Load Data


In [3]:
cell_df = pd.read_csv(r"C:\Users\ignat\Downloads\cell_samples.csv")
cell_df.head()


We check for the relationship among the classes based on Clump thickness and Uniformity of cell size:

In [ ]:
ax = cell_df[cell_df['Class'] == 4][0:50].plot(kind='scatter', x='Clump', y='UnifSize', color='DarkBlue', label='malignant');
cell_df[cell_df['Class'] == 2][0:50].plot(kind='scatter', x='Clump', y='UnifSize', color='Yellow', label='benign', ax=ax);
plt.show()

## Processing the data


In [ ]:
# We drop the non-numerical data in 'BareNuc' column
cell_df = cell_df[pd.to_numeric(cell_df['BareNuc'], errors='coerce').notnull()]
cell_df['BareNuc'] = cell_df['BareNuc'].astype('int')
cell_df.dtypes

In [ ]:
feature_df = cell_df[['Clump', 'UnifSize', 'UnifShape', 'MargAdh', 'SingEpiSize', 'BareNuc', 'BlandChrom', 'NormNucl', 'Mit']]
X = np.asarray(feature_df)
X[0:5]

In [ ]:
cell_df['Class'] = cell_df['Class'].astype('int')
y = np.asarray(cell_df['Class'])
y [0:5]

## Creating Train & Test dataset


In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)

# We check for the shape of the created datasets to prevent errors during model training resulting in different data shapes
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

#### Creating the model


In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='sigmoid')
clf.fit(X_train, y_train) 

After being fitted, the model can then be used to predict new values:


In [ ]:
yhat = clf.predict(X_test)
yhat [0:5]

## Model Evaluation

The confusion matrix is used since it provides great insights about the evaluation results
Then the f1_score & jaccard index, these present the score of the model in numerical value which gives a rough but easy-to-understand picture of how the model has performed

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, yhat, labels=[2,4])
np.set_printoptions(precision=2)

print (classification_report(y_test, yhat))

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Benign(2)','Malignant(4)'],normalize= False,  title='Confusion matrix')

You can also easily use the **f1\_score** & **jaccard_score** from sklearn library:


In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, yhat, average='weighted') 

In [ ]:
from sklearn.metrics import jaccard_score
jaccard_score(y_test, yhat,pos_label=2)

#### The SVM offers a choice of kernel functions for its processing
i.e. 
* as shown above **Radial Basis Function(rbf)**
* **linear**
* **polynomial**
* **sigmoid**

In order to get the right hyperameters i.e. the regularisation(determine the acceptable error), kernel function & gamma(determine the points considered to get the decision boundary), we perform hyperameter tuning as shown below

In [ ]:
from sklearn.model_selection import GridSearchCV 

param_grid = { 'kernel':['rbf', 'poly', 'sigmoid'], 'C':[0.1,1,10,100], 'gamma':[1,0.1,0.01,0.001]}

grid = GridSearchCV(svm.SVC(), param_grid, refit=True, verbose=2)
grid.fit(X_train, y_train)



In [ ]:
# We use the function below to get the parameters deemed best by the 'grid' method
print(grid.best_estimator_)

In [ ]:
# print(jaccard_score(y_test, , pos_label=2))
f1_score(y_test, grid.predict(X_test), average='weighted')